# Research Recommendation Agent - 추론(Inference) 노트북

**2025 DATA·AI 분석 경진대회 - 논문·데이터 추천 에이전트**

이 노트북은 Qwen3-14B 기반 연구 데이터/논문 추천 시스템의 추론을 수행합니다.

## 실행 환경
- GPU: NVIDIA RTX 3080 이상 (INT8 양자화 시 14GB VRAM)
- CUDA: 11.8+
- Python: 3.10+

## 실행 순서
1. 환경 변수 설정
2. 모델 로드
3. 추론 실행
4. 결과 확인

## 1. 환경 설정 및 라이브러리 임포트

In [1]:
import sys
import os
import logging

# 프로젝트 루트 경로를 명확하게 설정
# 이 노트북은 paper-reco-agent/notebooks/ 폴더에 위치
project_root = '/home/infidea/paper-reco-agent'

# sys.path에 프로젝트 루트 추가
sys.path.insert(0, project_root)

print(f"프로젝트 루트: {project_root}")
print(f"현재 작업 디렉토리: {os.getcwd()}")

# 로깅 설정 (주피터 셀에서 볼 수 있도록)
# 로그 디렉토리 생성
os.makedirs(os.path.join(project_root, 'logs'), exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),  # 주피터 셀에 출력
        logging.FileHandler(os.path.join(project_root, 'logs/app.log'))  # 파일에도 저장
    ],
    force=True  # 기존 로깅 설정 덮어쓰기
)

print(f"✅ 로깅 설정 완료 (콘솔 + 파일)")

# 필수 라이브러리 임포트
import asyncio
import json
from dotenv import load_dotenv

# 환경 변수 로드
env_path = os.path.join(project_root, '.env')
load_dotenv(env_path)
print(f"✅ 환경 변수 로드 완료")

프로젝트 루트: /home/infidea/paper-reco-agent
현재 작업 디렉토리: /home/infidea
✅ 로깅 설정 완료 (콘솔 + 파일)
✅ 환경 변수 로드 완료


## 2. GPU 및 CUDA 확인

In [2]:
import torch

# GPU 확인
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"사용 가능한 GPU 수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  GPU를 사용할 수 없습니다. CPU 모드 또는 DEV_MODE로 실행됩니다.")

PyTorch 버전: 2.8.0+cu128
CUDA 사용 가능: True
CUDA 버전: 12.8
사용 가능한 GPU 수: 1
현재 GPU: NVIDIA H100 80GB HBM3
GPU 메모리: 79.1 GB


## 3. 추천 에이전트 초기화

**중요**: 이 단계에서 Qwen3-14B 모델이 로드됩니다. GPU 메모리가 충분한지 확인하세요.
- INT4: ~8GB VRAM
- INT8: ~14GB VRAM
- FP16: ~28GB VRAM

In [3]:
# 추천 에이전트 임포트 및 초기화
from src.agents.recommendation_agent import KoreanResearchRecommendationAgent
from src.config.settings import settings

print("모델 설정:")
print(f"  - 모델명: {settings.MODEL_NAME}")
print(f"  - 양자화: {settings.QUANTIZATION}")
print(f"  - 임베딩 모델: {settings.EMBEDDING_MODEL}")
print(f"  - 개발 모드: {settings.DEV_MODE}")
print("\n🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)")

agent = KoreanResearchRecommendationAgent()

print("\n✅ 에이전트 초기화 완료")
print(f"모델 정보: {json.dumps(agent.llm_model.get_model_info(), indent=2, ensure_ascii=False)}")

/home/infidea/.conda/envs/paper-agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-10-14 01:34:29,245 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda:0
2025-10-14 01:34:29,247 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
모델 설정:
  - 모델명: Qwen/Qwen3-14B
  - 양자화: fp16
  - 임베딩 모델: intfloat/multilingual-e5-large
  - 개발 모드: False

🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)
2025-10-14 01:34:37,513 - src.agents.recommendation_agent - INFO - 🚀 프로덕션 모드로 실행: 실제 Qwen 모델 사용
False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
The following directories listed in your path were found to be non-existent: {PosixPath('//10.107.25.191'), PosixPath('tcp'), PosixPath('8888')}
The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Seoul')}
The following directori

/home/infidea/.conda/envs/paper-agent/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/home/infidea/.conda/envs/paper-agent/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/infidea/.conda/envs/paper-agent/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'), PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0')}.. We select the PyTorch default libcudart.so, which is {torch.version.cuda},but this might missmatch with the CUDA version that is needed for bitsandbytes.To override this behavior set the BNB_CUDA_V

2025-10-14 01:34:38,279 - src.models.qwen_model - INFO -    - FP16 모드 (~28GB VRAM)


Loading checkpoint shards: 100%|██████████| 8/8 [00:13<00:00,  1.69s/it]


2025-10-14 01:35:05,204 - src.models.qwen_model - INFO - ✅ Qwen 모델 로딩 완료

✅ 에이전트 초기화 완료
모델 정보: {
  "model_name": "Qwen/Qwen3-14B",
  "device": "cuda",
  "quantization": "fp16",
  "max_tokens": 512,
  "temperature": 0.1,
  "parameters": "14.8B",
  "context_length": "32K (extendable to 128K)"
}


## 4. 테스트 데이터셋 ID 설정

DataON에 등록된 실제 데이터셋 ID를 입력하세요.

In [4]:
# 테스트용 데이터셋 ID
# 예시: KISTI DataON의 실제 데이터셋 ID를 입력하세요
test_dataset_id = "c7dc77b406795dcc332dcc733efb2261"  # TODO: 실제 데이터셋 ID로 변경

print(f"테스트 데이터셋 ID: {test_dataset_id}")

테스트 데이터셋 ID: c7dc77b406795dcc332dcc733efb2261


## 5. 추론 실행

에이전트가 다음 단계를 수행합니다:
1. 소스 데이터셋 메타데이터 조회 (DataON API)
2. LLM으로 검색 쿼리 생성
3. 후보 수집 (DataON + ScienceON API)
4. 하이브리드 유사도 계산 (E5 + BM25)
5. LLM으로 최종 추천 생성

In [5]:
# 추론 실행 (비동기)
import time

start_time = time.time()
print("🔍 추천 시작...\n")

# Jupyter에서 비동기 함수 실행
result = await agent.recommend(test_dataset_id)

elapsed_time = time.time() - start_time
print(f"\n✅ 추천 완료! (소요 시간: {elapsed_time:.2f}초)")

🔍 추천 시작...

2025-10-14 01:35:13,897 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID c7dc77b406795dcc332dcc733efb2261
2025-10-14 01:35:14,013 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/c7dc77b406795dcc332dcc733efb2261?key=4936BC43D48603524DEDA2E2D56D6B46 "HTTP/1.1 200 200"
2025-10-14 01:35:14,015 - src.clients.dataon_client - INFO - Successfully retrieved metadata for dataset c7dc77b406795dcc332dcc733efb2261
2025-10-14 01:35:14,016 - src.clients.dataon_client - INFO - API Response:
{
  "response": {
    "elapsed time": "37 ms",
    "status": "200",
    "message": "OK",
    "total count": "1",
    "type": "json"
  },
  "records": {
    "svc_id": "c7dc77b406795dcc332dcc733efb2261",
    "ctlg_type": "02",
    "dataset_type": "01",
    "ctlg_type_pc": "dataset",
    "dataset_type_pc": "국내",
    "dataset_pub_dt_pc": "2025",
    "dataset_access_type_pc": "공개",
    "file_yn_pc": "랜딩페이지이동",
    "dataset_cc_license_pc": "none",
    "datas

Batches: 100%|██████████| 1/1 [00:00<00:00, 69.24it/s]

2025-10-14 01:35:23,752 - src.agents.recommendation_agent - INFO - 상위 15개 후보 순위 결정 완료
2025-10-14 01:35:23,753 - src.agents.recommendation_agent - INFO - LLM 추천 생성 시도 1/2


2025-10-14 01:35:30,294 - src.agents.recommendation_agent - INFO - 추출된 JSON:
{
  "recommendations": [
    {
      "candidate_number": 1,
      "reason": "Shared focus on targeted metabolomics and metabolic mechanisms related to disease, with common terms like '대사체' and '표적'",
      "level": "참고"
    },
    {
      "candidate_number": 3,
      "reason": "Involves metabolomics research methods and shares common terms such as '대사체' and '대사체학', relevant to the study's methodology",
      "level": "참고"
    },
    {
      "candidate_number": 5,
      "reason": "Utilizes metabolomics techniques and mentions plasma ('혈장'), aligning with the source dataset’s focus on plasma metabolite analysis",
      "level": "참고"
    },
    {
      "candidate_number": 6,
      "reason": "Shares the keyword '대사체' and uses NMR technology, which is similar to the source dataset’s analytical method",
      "level": "참고"
    },
    {
      "candidate_number": 9,
      "reason": "Includes the term 'BDS' which is pa

## 6. 결과 확인

In [ ]:
# 오류 확인
if 'error' in result:
    print(f"❌ 오류 발생: {result['error']}")
else:
    print("=" * 80)
    print("📊 추천 결과 요약")
    print("=" * 80)
    print(f"\n소스 데이터셋:")
    print(f"  ID: {result['source_dataset']['id']}")
    print(f"  제목: {result['source_dataset']['title']}")
    print(f"  키워드: {', '.join(result['source_dataset']['keywords'])}")
    
    print(f"\n추천 개수: {len(result['recommendations'])}개")
    print(f"분석 후보: {result['candidates_analyzed']}개")
    print(f"처리 시간: {result['processing_time_ms']}ms")
    
    print(f"\n모델 정보:")
    for key, value in result['model_info'].items():
        print(f"  {key}: {value}")
    
    print("\n" + "=" * 80)
    print("📝 추천 목록")
    print("=" * 80)

In [ ]:
# 추천 목록 상세 출력
if 'recommendations' in result:
    for rec in result['recommendations']:
        print(f"\n[{rec['rank']}위] {rec['level']} - {rec['type'].upper()}")
        print(f"제목: {rec['title']}")
        print(f"점수: {rec['score']:.3f}")
        print(f"이유: {rec['reason']}")
        print(f"URL: {rec['url']}")
        print("-" * 80)

## 7. JSON 파일로 결과 저장

In [ ]:
# 결과를 JSON 파일로 저장
output_dir = os.path.join(project_root, 'data', 'inference_results')
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, f"result_{test_dataset_id}.json")

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"✅ 결과 저장 완료: {output_file}")

## 8. 리소스 정리

In [ ]:
# GPU 메모리 정리
if hasattr(agent, 'llm_model') and agent.llm_model:
    agent.llm_model.cleanup()
    print("✅ 모델 리소스 정리 완료")

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ GPU 메모리 캐시 정리 완료")

## 9. 추가 테스트 (선택사항)

여러 데이터셋에 대해 배치 추론을 수행할 수 있습니다.

In [ ]:
# 여러 데이터셋 ID 배치 테스트
test_dataset_ids = [
    "DATASET_ID_1",
    "DATASET_ID_2",
    "DATASET_ID_3"
]

batch_results = []

for dataset_id in test_dataset_ids:
    print(f"\n처리 중: {dataset_id}")
    try:
        result = await agent.recommend(dataset_id)
        batch_results.append({
            'dataset_id': dataset_id,
            'success': 'error' not in result,
            'result': result
        })
        print(f"✅ 완료: {len(result.get('recommendations', []))}개 추천")
    except Exception as e:
        print(f"❌ 실패: {e}")
        batch_results.append({
            'dataset_id': dataset_id,
            'success': False,
            'error': str(e)
        })

# 배치 결과 저장
batch_output_file = os.path.join(output_dir, 'batch_results.json')
with open(batch_output_file, 'w', encoding='utf-8') as f:
    json.dump(batch_results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 배치 결과 저장 완료: {batch_output_file}")